## Notebook to look at trends in cell velocity

In [1]:
def cartesian_to_spherical(x, y, z, center=(0, 0, 0)):
    """
    Convert Cartesian coordinates (x, y, z) to spherical coordinates (r, theta, phi).
    
    Parameters:
    - x, y, z: Arrays or scalars of Cartesian coordinates.
    - center: Tuple (x_c, y_c, z_c) representing the center of the sphere.
    
    Returns:
    - r: Radial distance from the center.
    - theta: Colatitude angle in radians (0 to pi).
    - phi: Longitude angle in radians (0 to 2pi).
    """
    x_c, y_c, z_c = center
    
    # Shift coordinates relative to the center
    x_rel = x - x_c
    y_rel = y - y_c
    z_rel = z - z_c
    
    # Compute spherical coordinates
    r = np.sqrt(x_rel**2 + y_rel**2 + z_rel**2)          # Radial distance
    theta = np.arccos(z_rel / r)                        # Colatitude
    phi = np.arctan2(y_rel, x_rel)                      # Longitude (0 to 2pi)
    
    return r, theta, phi

In [2]:
import os
import pandas as pd
import numpy as np
from src.utilities.shape_utils import plot_mesh
from scipy.spatial import KDTree
import plotly.express as px
import trimesh

# Load test dataset that used Kikume NLS marker
root = "E:\\Nick\\Cole Trapnell's Lab Dropbox\\Nick Lammers\\Nick\\killi_tracker\\"
experiment_date = "20240611_NLS-Kikume_24hpf_side2"
config_name = "tracking_jordao_20240918.txt"
model ="LCP-Multiset-v1"
tracking_folder = config_name.replace(".txt", "")
tracking_folder = tracking_folder.replace(".toml", "")

well_num = 0
start_i = 0
stop_i = 1600

suffix = ""

# get path to metadata
metadata_path = os.path.join(root, "metadata", "tracking")

# set output path for tracking results
project_path = os.path.join(root, "tracking", experiment_date,  tracking_folder, f"well{well_num:04}" + suffix, "")
project_sub_path = os.path.join(project_path, f"track_{start_i:04}" + f"_{stop_i:04}" + suffix, "")

# load the tracks 
deep_tracks_df = pd.read_csv(os.path.join(project_sub_path, "deep_tracks_df.csv"))

# load velocity info
cell_vel_df = pd.read_csv(os.path.join(project_sub_path, "cell_velocity_df.csv"))

# load sphere mesh
sphere_mesh = trimesh.load(os.path.join(project_sub_path, "embryo_sphere_mesh.obj"))

### Calculate velocity magnitude at vertices of a lower-res sphere

In [4]:
f, v = sphere_mesh.faces.copy(), sphere_mesh.vertices.copy()

# sum velocity components
dim_vec = cell_vel_df["dim"].to_numpy()[:, None]
time_vec = cell_vel_df.loc[dim_vec==1, "time"].to_numpy()[:, None]
vf_cols = [f"vel_f{i:06}" for i in range(f.shape[0])]

vx_df = cell_vel_df.loc[dim_vec==0, :].copy()
vy_df = cell_vel_df.loc[dim_vec==1, :].copy()
vz_df = cell_vel_df.loc[dim_vec==2, :].copy()

# v_array = np.sqrt(vx_df[vf_cols].to_numpy()**2 + vy_df[vf_cols].to_numpy()**2 + vz_df[vf_cols].to_numpy()**2)
# cell_speed_df = pd.DataFrame(np.c_[time_vec, v_array], columns=["time"] + vf_cols)
vx_df.head()

,time,dim,vel_f000000,vel_f000001,vel_f000002,vel_f000003,vel_f000004,vel_f000005,vel_f000006,vel_f000007,...,vel_f020470,vel_f020471,vel_f020472,vel_f020473,vel_f020474,vel_f020475,vel_f020476,vel_f020477,vel_f020478,vel_f020479
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [48]:
# smooth
# get moving average for each face over time
avg_window = 15
vx_df_sm = vx_df.copy()
vx_df_sm[vf_cols] = vx_df_sm[vf_cols].rolling(center=True, window=avg_window, min_periods=1).mean()

vy_df_sm = vy_df.copy()
vy_df_sm[vf_cols] = vy_df_sm[vf_cols].rolling(center=True, window=avg_window, min_periods=1).mean()

vz_df_sm = vz_df.copy()
vz_df_sm[vf_cols] = vz_df_sm[vf_cols].rolling(center=True, window=avg_window, min_periods=1).mean()

In [49]:
from src.utilities.shape_utils import calculate_face_centroids

# calculate face centroids
face_centroids = calculate_face_centroids(sphere_mesh)

fx, fy, fz = face_centroids[:, 0], face_centroids[:, 1], face_centroids[:, 2]
C = np.mean(face_centroids, axis=0)
radius = np.linalg.norm(face_centroids[0, :] - C)

# get lower-resolution mesh to do spatial averaging
sphere_mesh_raw = trimesh.creation.icosphere(subdivisions=4, radius=1.0)

verts = np.asarray(sphere_mesh_raw.vertices.copy())
verts = verts * radius
verts = verts + C[None, :]

sphere_mesh_lr = sphere_mesh_raw.copy()
sphere_mesh_lr.vertices = verts
verts.shape

(2562, 3)

In [50]:
# take average value in the neighborhood of each vertex
from scipy.spatial import KDTree
from tqdm import tqdm
from scipy.spatial import distance_matrix

# get faces and vertices
flr, vlr = sphere_mesh_lr.faces.copy(), sphere_mesh_lr.vertices.copy()

# map high-res faces to low-res vertices
# tree = KDTree(vlr)
# vlr_distances, vlr_indices = tree.query(face_centroids)
p_dist_mat = distance_matrix(vlr, face_centroids)

In [51]:
radius = 30

# get full speed array
vx_array_full = vx_df_sm[vf_cols].to_numpy()
vy_array_full = vy_df_sm[vf_cols].to_numpy()
vz_array_full = vz_df_sm[vf_cols].to_numpy()

# initialize down-sampled array
vx_array = np.zeros((vx_array_full.shape[0], vlr.shape[0]))
vy_array = np.zeros((vx_array_full.shape[0], vlr.shape[0]))
vz_array = np.zeros((vx_array_full.shape[0], vlr.shape[0]))

for v in tqdm(range(vx_array.shape[1])):
    dist_vec = p_dist_mat[v, :]
    vx_array[:, v] = np.mean(vx_array_full[:, dist_vec < radius], axis=1)
    vy_array[:, v] = np.mean(vy_array_full[:, dist_vec < radius], axis=1)
    vz_array[:, v] = np.mean(vz_array_full[:, dist_vec < radius], axis=1)

100%|██████████| 2562/2562 [00:01<00:00, 1729.91it/s]


In [53]:
import plotly.graph_objects as go
from tqdm import tqdm
from src.utilities.plot_utils import mesh_face_plot
from scipy.interpolate import griddata


cl = 0 # + 0.04#+ density_baseline
cu = 7.5 #+ 0.04# + density_baseline
frame_path = os.path.join(project_sub_path, "cone_frame2s", "")
os.makedirs(frame_path, exist_ok=True)

max_v = np.percentile(np.sqrt(vx_array**2 + vy_array**2 + vz_array**2), 99.9)

time_vec = cell_speed_df["time"].to_numpy()
offset = 3
plot_interval = 5

for t in tqdm(range(0, int(time_vec[-1]), plot_interval)):
    
    plot_time = time_vec[t]

    camera=dict(
                eye=dict(x=1, y=1, z=1),  # Camera position
                up=dict(x=0, y=0, z=1),     # "Up" direction
    #             center=dict(x=0, y=0, z=0)  # Focal point
            )


    #########
    # Sphere surface
    # Extract x, y, z coordinates from vertices
    x, y, z = vlr[:, 0], vlr[:, 1], vlr[:, 2]

    # Extract i, j, k indices from faces
    i, j, k = flr[:, 0], flr[:, 1], flr[:, 2]

    # flux
    u, v, w = vx_array[t, ].copy().T, vy_array[t, ].copy().T, vz_array[t, ].copy().T

    # Normalize vectors for consistent length
    magnitude = np.sqrt(u**2 + v**2 + w**2)
#     max_mag = np.max(magnitude)
    target_mag = magnitude / max_v + offset
    factor = np.divide(target_mag, magnitude + 1e-6) 
    u = np.multiply(u, factor)
    v = np.multiply(v, factor)
    w = np.multiply(w, factor)

    # # Scale vectors for visualization
    # scale = 0.1
    # u *= scale
    # v *= scale
    # w *= scale

    # Define arrow origins (shifted slightly outward for visibility)
    x_start = x
    y_start = y
    z_start = z

    # Define arrow tips (add vectors to starting points)
    x_end = x_start + u
    y_end = y_start + v
    z_end = z_start + w

    # Flatten arrays for plotting
    x_start, y_start, z_start = x_start.flatten(), y_start.flatten(), z_start.flatten()
    x_end, y_end, z_end = x_end.flatten(), y_end.flatten(), z_end.flatten()
    u, v, w = u.flatten(), v.flatten(), w.flatten()

    # fig = mesh_face_plot(f, v, vel_vec, colormap="RdBu_r") #"deep_r")
    fig = go.Figure(data=[go.Mesh3d(
            x=x, y=y, z=z,            # Vertex coordinates
            i=i, j=j, k=k,
    #         intensity=vel_vec,
            flatshading=True,
            colorscale="RdBu_r", 
        opacity=0.5
        )])


    ###########
    # Flux cones 

    # Plot vector field using cones
    fig.add_trace(go.Cone(
        x=x_start, y=y_start, z=z_start,
        u=u, v=v, w=w,
    #     sizemode="absolute",
    #     sizeref=0.1,
        anchor="center",
        cmin=offset,            # Minimum value for color scaling
        cmax=offset+1,
        colorbar=dict(title='Velocity Magnitude'),  # Add colorbar
        sizemode="scaled",  # Scale size with magnitude
        sizeref=0.8,  # Adjust cone size scaling
        showscale=True,  # Show colorbar
    ))

    # Layout settings
    fig.update_layout(
        title="Cell Velocity Field",
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode="data"
        )
    )


    fig.update_layout(
        scene=dict(camera=dict(
                eye=dict(x=0, y=0.35, z=2),  # Camera position
                up=dict(x=0, y=-1, z=0),     # "Up" direction
                center=dict(x=0, y=0, z=0)  # Focal point
            ),
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    fig.update_traces(lightposition=dict(
            x=5,  # Position the light source
            y=5,
            z=5
        ), 
            lighting=dict(
            ambient=0.85,    # Soft overall lighting
            diffuse=0.2,    # Scattered light
            specular=0,   # Shiny reflections
            roughness=0.2,  # Smooth surface
            fresnel=0     # Glow at edges
        ),

                    )

    fig.update_traces(showscale=False)
# fig.show()
    fig.write_image(frame_path + f"cone_frame{int(plot_time):04}.png", scale=1)

100%|██████████| 320/320 [09:07<00:00,  1.71s/it]


### Hard to parse. Let's look at divergence

In [99]:
radius = 75

# get full speed array
vx_array_full = vx_df_sm[vf_cols].to_numpy()
vy_array_full = vy_df_sm[vf_cols].to_numpy()
vz_array_full = vz_df_sm[vf_cols].to_numpy()

# initialize down-sampled array
div_array = np.zeros((vx_array_full.shape[0], vlr.shape[0]))

for vi in tqdm(range(vx_array.shape[1])):
    dist_vec = p_dist_mat[vi, :]
    dist_filter =  dist_vec < radius

    # get velocity (numerator)
    vx_vec = vx_array_full[:, dist_filter]
    vy_vec = vy_array_full[:, dist_filter]
    vz_vec = vz_array_full[:, dist_filter]

    dv = np.concatenate((vx_vec[:, :, None], vy_vec[:, :, None], vz_vec[:, :, None]), axis=2)
    dv = np.transpose(dv, (1, 2, 0))
    # get dist changes (denominator)
    dr = np.sign(face_centroids[dist_filter, :] - vlr[vi, :])

    derivatives = np.divide(dv, dr[:, :, None])
    divs = np.squeeze(np.sum(derivatives, axis=1)).T
    div_array[:, vi] = np.mean(divs, axis=1)


100%|██████████| 2562/2562 [00:45<00:00, 56.29it/s]


In [90]:
vi = np.argmax(vlr[:, 2])

dist_vec = p_dist_mat[vi, :]
dist_filter =  dist_vec < radius

# get velocity (numerator)
vx_vec = vx_array_full[:, dist_filter]
vy_vec = vy_array_full[:, dist_filter]
vz_vec = vz_array_full[:, dist_filter]

dv = np.concatenate((vx_vec[:, :, None], vy_vec[:, :, None], vz_vec[:, :, None]), axis=2)
dv = np.transpose(dv, (1, 2, 0))
# get dist changes (denominator)
dr = np.signface_centroids[dist_filter, :] - vlr[vi, :]

derivatives = np.divide(dv, dr[:, :, None])
divs = np.squeeze(np.sum(derivatives, axis=1)).T
# div_array[:, v] = np.mean(divs, axis=1)

In [105]:
t = 1500


# fig = mesh_face_plot(f, v, vel_vec, colormap="RdBu_r") #"deep_r")
frame_path = os.path.join(project_sub_path, "div_frames", "")
os.makedirs(frame_path, exist_ok=True)

for t in tqdm(range(0, 1600)):
    
    plot_time = time_vec[t]
    
    div_vec = div_array[t, :]
    
    fig = go.Figure(data=[go.Mesh3d(
            x=x, y=y, z=z,            # Vertex coordinates
            i=i, j=j, k=k,
            intensity=div_vec,
            flatshading=True,
            colorscale="RdBu_r", 
        opacity=1
        )])


    #     ###########
    #     # Flux cones 

    #     # Plot vector field using cones
    #     fig.add_trace(go.Cone(
    #         x=x_start, y=y_start, z=z_start,
    #         u=u, v=v, w=w,
    #     #     sizemode="absolute",
    #     #     sizeref=0.1,
    #         anchor="center",
    #         cmin=offset,            # Minimum value for color scaling
    #         cmax=offset+1,
    #         colorbar=dict(title='Velocity Magnitude'),  # Add colorbar
    #         sizemode="scaled",  # Scale size with magnitude
    #         sizeref=0.8,  # Adjust cone size scaling
    #         showscale=True,  # Show colorbar
    #     ))

    # Layout settings
    fig.update_layout(
        title="Cell Velocity Field",
        scene=dict(
            xaxis=dict(visible=False),
            yaxis=dict(visible=False),
            zaxis=dict(visible=False),
            aspectmode="data"
        )
    )


    fig.update_layout(
        scene=dict(camera=dict(
                eye=dict(x=0, y=0.35, z=2),  # Camera position
                up=dict(x=0, y=-1, z=0),     # "Up" direction
                center=dict(x=0, y=0, z=0)  # Focal point
            ),
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z'
        )
    )

    fig.update_traces(lightposition=dict(
            x=5,  # Position the light source
            y=5,
            z=5
        ), 
            lighting=dict(
            ambient=0.85,    # Soft overall lighting
            diffuse=0.2,    # Scattered light
            specular=0,   # Shiny reflections
            roughness=0.2,  # Smooth surface
            fresnel=0     # Glow at edges
        ),

                    )
    
    fig.update_traces(cmin=-2.5, cmax=2.5, showscale=False)
# fig.show()
    fig.write_image(frame_path + f"div_frame{int(plot_time):04}.png", scale=1)

#     fig.show()

100%|██████████| 1600/1600 [30:09<00:00,  1.13s/it]


In [73]:
test = np.concatenate((vx_vec[:, :, None], vy_vec[:, :, None], vz_vec[:, :, None]), axis=2)
test = np.transpose(test, (1, 2, 0))
test.shape

(50, 3, 1600)